In [1]:
import sys
sys.path.append('/home/dbswldms316/FLamby/')

from flamby.datasets.fed_tcga_brca.dataset import TcgaBrcaRaw, FedTcgaBrca
from torch.utils.data import Dataset, DataLoader

mydataset_raw = TcgaBrcaRaw()
mydataset_pooled = FedTcgaBrca(train=True, pooled=True)

from flamby.datasets.fed_tcga_brca import (
    BATCH_SIZE,
    LR,
    NUM_EPOCHS_POOLED,
    Baseline,
    BaselineLoss,
    metric,
    NUM_CLIENTS,
    Optimizer,
)

# class DictDataset(Dataset):
#     def __init__(self, data):
#         self.data = data
    
#     def __len__(self):
#         return len(self.keys)
    
#     def __getitem__(self, idx):
#         key = self.keys[idx]
#         return self.data_dict[key]
    


In [2]:
import torch
# Instantiation of local train set (and data loader)), baseline loss function, baseline model, default optimizer
#feature_tensor = torch.tensor([[1,10], [2,20], [3,30], [4,40], [5,50], [6,60], [7,70], [8,80],[9,90],[10,100]])
for i in [0,1,2,3,4,5]:
    train_dataset = FedTcgaBrca(center=i, train=True, pooled=False)
    print(train_dataset[0])
    age_groups = {}
    for person in train_dataset:
        age = int(person[0][0])
        age_group = int(age - age % 10)
        if age_group not in age_groups:
            age_groups[age_group] = []
        age_groups[age_group].append(person)
    age_groups = dict(sorted(age_groups.items()))
    
    result_dict = {'0 to 30':[], '40 to 60':[], '70 to 100':[]}
    for key in age_groups.keys():
        if 0 < key <=30:
            result_dict['0 to 30']+=(age_groups[key])
        elif 30 < key <= 70:
            result_dict['40 to 60']+=(age_groups[key])
        else:
            result_dict['70 to 100']+=(age_groups[key])
    print('** train dataset includes age groups like: ', [len(age_groups[i]) for i in age_groups.keys()])
    #print(result_dict)
    print('** train dataset after split: ', [len(result_dict[i]) for i in result_dict.keys()])
    # train_dataset = DictDataset(result_dict)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=0)
    #train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)

    
    lossfunc = BaselineLoss()
    model = Baseline()
    optimizer = Optimizer(model.parameters(), lr=LR)
    
    
    # Traditional pytorch training loop
    for epoch in range(0, NUM_EPOCHS_POOLED):
        for X,y in train_dataloader:
            optimizer.zero_grad()
            outputs = model(X)
            loss = lossfunc(outputs, y)
            loss.backward()
            optimizer.step()
        
    # Evaluation
    from flamby.utils import evaluate_model_on_tests
    # Instantiation of a list of the local test sets
    test_dataset = FedTcgaBrca(center=i, train=False, pooled=False)
    age_groups = {}
    for person in test_dataset:
        age = int(person[0][0])
        age_group = age - age % 10
        if age_group not in age_groups:
            age_groups[age_group] = []
        age_groups[age_group].append(person)
    age_groups = dict(sorted(age_groups.items()))
    
    result_dict = {'0 to 30': [], '31 to 70': [], '71 to 100': []}
    for key in age_groups.keys():
        if key <= 30:
            result_dict['0 to 30'] += age_groups[key]
        elif key <= 70:
            result_dict['31 to 70'] += age_groups[key]
        else:
            result_dict['71 to 100'] += age_groups[key]

    print('** test dataset includes age groups like: ',[len(age_groups[i]) for i in age_groups.keys()])
    print('** test dataset after split: ', [len(result_dict[i]) for i in result_dict.keys()])
    
    # 각 나이 범주에 해당하는 데이터셋 생성
    test_dataset_0_to_30 = result_dict['0 to 30']
    test_dataset_40_to_60 = result_dict['31 to 70']
    test_dataset_70_to_100 = result_dict['71 to 100']

    # 각 데이터셋을 DataLoader로 변환
    test_dataloader_0_to_30 = torch.utils.data.DataLoader(test_dataset_0_to_30,  shuffle=False)
    test_dataloader_40_to_60 = torch.utils.data.DataLoader(test_dataset_40_to_60,  shuffle=False)
    test_dataloader_70_to_100 = torch.utils.data.DataLoader(test_dataset_70_to_100,  shuffle=False)
    print(next(iter(test_dataloader_0_to_30)))
    # evaluate_model_on_tests 함수 호출
    with torch.no_grad():
        dict_cindex_0_to_30 = evaluate_model_on_tests(model, [test_dataloader_0_to_30], metric)
        dict_cindex_40_to_60 = evaluate_model_on_tests(model, [test_dataloader_40_to_60], metric)
        dict_cindex_70_to_100 = evaluate_model_on_tests(model, [test_dataloader_70_to_100], metric)

    # 결과 출력
    print("Results for age group 0 to 39:", dict_cindex_0_to_30)
    print("Results for age group 41 to 79:", dict_cindex_40_to_60)
    print("Results for age group 80 to 100:", dict_cindex_70_to_100)


/home/dbswldms316/FLamby/flamby/datasets/fed_tcga_brca/dataset.py:52: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return (torch.tensor(x, dtype=self.X_dtype), torch.tensor(y, dtype=self.y_dtype))


(tensor([60.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  1.,
         0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.]), tensor([  1., 921.]))
** train dataset includes age groups like:  [2, 17, 52, 66, 60, 39, 12]
** train dataset after split:  [19, 217, 12]


/home/dbswldms316/FLamby/flamby/datasets/fed_tcga_brca/dataset.py:52: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return (torch.tensor(x, dtype=self.X_dtype), torch.tensor(y, dtype=self.y_dtype))
/home/dbswldms316/FLamby/flamby/datasets/fed_tcga_brca/dataset.py:52: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return (torch.tensor(x, dtype=self.X_dtype), torch.tensor(y, dtype=self.y_dtype))


** test dataset includes age groups like:  [1, 5, 14, 11, 19, 7, 4, 2]
** test dataset after split:  [6, 51, 6]
[tensor([[29.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
          0.,  1.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  1.,  1.,  0.,  1.,
          0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.]]), tensor([[   0., 3286.]])]


100%|██████████| 1/1 [00:00<00:00, 910.82it/s]
/home/dbswldms316/FLamby/flamby/datasets/fed_tcga_brca/dataset.py:52: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return (torch.tensor(x, dtype=self.X_dtype), torch.tensor(y, dtype=self.y_dtype))


Results for age group 0 to 39: {'client_test_0': 0.5}
Results for age group 41 to 79: {'client_test_0': 0.5955056179775281}
Results for age group 80 to 100: {'client_test_0': 0.25}
(tensor([63.,  1.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,
         0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  1.,  0.,
         1.,  1.,  1.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.]), tensor([  0., 343.]))
** train dataset includes age groups like:  [1, 8, 30, 47, 43, 22, 4, 1]
** train dataset after split:  [9, 142, 5]


KeyboardInterrupt: 